# Taller RCT con covariable

En este ejercicio se hará la inferencia de los ATE con y sin covariables pre-tratamiento. De esta manera analizamos tanto la presencia de efectos heterogéneos así como el cambio en la precisión del estimador del ATE. Para ello seguiremos de cerca el notebook [Reemployment Bonus RCT](https://colab.research.google.com/github/CausalAIBook/MetricsMLNotebooks/blob/main/CM1/python-rct-penn-precision-adj.ipynb)

## Datos

Se usan los datos del expermimento *Pensylvania reemployment bonus experiment*. En este experimento, los solicitantes de seguro de desempleo fueron asignados aleatoriamente a un grupo de control o a alguno de seis grupos de tratamiento. El tratamiento consiste en la oferta de un bono en efectivo si los individuos consiguen un empleo en un periodo de tiempo pre-especificado y que el trabajo fuese mantenido por un periodo de tiempo. 

Para una descripción del experimento, Revise el artículo: Bilias, Y. (2000). Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment. Journal of Applied Econometrics, 15(6), 575-594.

| Grupo | Bono | Periodo de calificación | oferta de taller
|-------|------|-------------------------|------------------|
| 0     | 0    | 0                       | No               |
| 1     | Low  | Short                   | Yes              |
| 2     | Low  | Long                    | Yes              |
| 3     | High | Short                   | Yes              |
| 4     | High | Long                    | Yes              |
| 5     | High, declining  | Long                    | Yes              |
| 6     | High  | Long                    | No              |

Donde bono low es en promedio $500 y high $997. Para el periodo de calificación short corresponde a 6 semanas y largo a 12 semanas

A continuación se importan los datos y se trabaja con el grupo de tratamiento 4

In [ ]:
pip install numpy pandas statsmodels patsy

In [ ]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", sep='\\s+')
n, p = data.shape
data = data[data["tg"].isin([0, 4])]

In [ ]:
data["T4"] = np.where(data["tg"] == 4, 1, 0)
data["T4"].value_counts()

In [ ]:
data.describe()

## Modelo

Sea $Y$ la duración del desempleo en log, $D$ la variable indicadora de tratamiento y $W$ un conjunto de variables de control, entonces en el modelo de regresión

\begin{equation*}
Y=\beta_1D+W'\boldsymbol{\beta_2}+\epsilon
\end{equation*}

el coeficiente $\beta_1$ es el ATE, si asumimos que $\epsilon$ no está correlacionado con $D$ ni $W$. Esto gracias a la asignación aleatoria al tratamiento. 

### Balanceo de covariables



Primero examinamos las columnas de la matriz de diseño, matirz con todas las covariables, a fin de mantener las que son linealmente independientes. Eliminar variables con alta colinealidad. Se usa la descomposición QR

In [ ]:
formula = ("0 ~ "
           "(female + black + othrace + C(dep) + q2 + q3 + q4 + q5 + q6 "
           "+ agelt35 + agegt54 + durable + lusd + husd)**2")
X = patsy.dmatrix(formula, data=data, return_type='dataframe')
X.shape

In [ ]:
def qr_decomposition(x):
    # Get a set of columns with no linear dependence for smallest subset of observations
    Q, Rx = np.linalg.qr(x)
    ex = np.abs(np.diag(Rx))
    keep = np.where(ex > 1e-6)[0]
    xS = x.iloc[:, keep]

    return xS


xS = qr_decomposition(X)
xS.shape

In [ ]:
covariate_balance = sm.OLS(data["T4"], xS)
cb_fit = covariate_balance.fit(cov_type="HC1", use_t=True)
print(f"R2:{cb_fit.rsquared}")
#cb_fit.summary()


Recuerde que la condición de balanceo de covariables implica que la regresión de $D$ sobre $W$ debe arrojar un $R^2=0$. Acá observamos un $R^2=0.029$. También esperaríamos que en las hipótesis de insignificancia individual no se rechazaran. Para probar esto debemos tener en cuenta que tenemos más de 100 regresores, luego al realizar multiples pruebas de hipótesis la probabilidad de error tipo I se incrementa a medida que aumentan las pruebas realizadas. Para corregirlo se usa el método de Holm-Bonferroni

In [ ]:
def holm_bonferroni(p, alpha=0.05):

    n = len(p)
    sig_beta = []

    for i in range(n):
        if np.sort(p)[i] > alpha / (n - i):
            break
        else:
            sig_beta.append(np.argsort(p).iloc[i])
            i += 1

    return sig_beta


print("Significant Coefficients (Indices): ", holm_bonferroni(cb_fit.pvalues, alpha=0.05))

De los resultados anteriores concluimos que las condiciones de balanceo no se cumplen a cabalidad

### Estimación

Empezamos por el modelo sin covariables $Y=\beta_1D+\epsilon$. El ATE es el coeficiente $\beta_1$.

In [ ]:
cl=smf.ols("np.log(inuidur1)~T4", data=data)
cl_results=cl.fit(cov_type="HC1")
beta1=cl_results.params["T4"]
se_beta1=cl_results.bse["T4"]
print(f"ATE:{beta1:.4f} , (se:{se_beta1:.4f})")

Ahora agregamos las covariables al modelo

In [ ]:
m_cov=('np.log(inuidur1)~T4+(female+black+othrace+agelt35+agegt54)')
cra=smf.ols(m_cov, data=data)
cra_results=cra.fit(cov_type="HC1")
beta1_cov=cra_results.params["T4"]
se_beta1_cov=cra_results.bse["T4"]
print(f"ATE with covariates:{beta1_cov:.4f} , (se:{se_beta1_cov:.4f})")

In [ ]:
m_cov=('np.log(inuidur1)~T4*(female+black+othrace+agelt35+agegt54)')
cra=smf.ols(m_cov, data=data)
cra_results=cra.fit(cov_type="HC1")
print(cra_results.summary())
beta1_cov=cra_results.params["T4"]
se_beta1_cov=cra_results.bse["T4"]
print(f"ATE with covariates:{beta1_cov:.4f} , (se:{se_beta1_cov:.4f})")

Agregamos interacciones para mejorar la precisión del estimador $E[Y|D,W]=\alpha ' XD+\beta ´X$, donde $X=(1,W)'$. Asumimos además $E[W]=0$, es decir que centramos las covariables

In [ ]:
ira_formula = "(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2"
X = patsy.dmatrix(ira_formula, data, return_type='dataframe')
X.head()

In [ ]:
X.columns = [f'x{t}' for t in range(X.shape[1])]  # clean column names
X = (X - X.mean(axis=0))  # demean all control covariates

# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula = "T4 * (" + "+".join(X.columns) + ")"
X['T4'] = data['T4']
X = patsy.dmatrix(ira_formula, X, return_type='dataframe')
X.head()


In [ ]:
ira = sm.OLS(np.log(data[["inuidur1"]]), X)
ira_results = ira.fit(cov_type="HC0")
ira_est = ira_results.params['T4']

interaction_cols = list(map(lambda x: x.startswith('T4:'), list(X.columns)))
cate = X.iloc[:, interaction_cols] @ ira_results.params[interaction_cols]
ira_se = np.sqrt(ira_results.HC0_se['T4']**2 + np.var(cate) / X.shape[0])

print(f"ATE: {ira_est:.4f}, (std={ira_se:.4f})")

## Ejercicio

Realice nuevamente el ejercicio, pero analizando el tratamiento 2. Compare con los resultados obtenidos anteriormente. 